<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q matplotlib-venn

In [0]:
!pip install mne

    100% |████████████████████████████████| 6.2MB 5.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/39/f9/f5/3e2fdef321f8e1f64061730c8ba2c81ad55c4b1b860b29d2dd
Successfully built mne


In [0]:
import mne

In [0]:
!pip install -U mne

Requirement already up-to-date: mne in /usr/local/lib/python3.6/dist-packages (0.17.1)


In [0]:
!pip install -U https://api.github.com/repos/mne-tools/mne-python/zipball/master

     \ 84.8MB 86.8MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-ed7ybs5g/wheels/45/5a/19/24fcd36c15fdecdbfdbe6dba22e0bacac8430a0e0274e9e052
Successfully built mne
  Found existing installation: mne 0.18.dev0
    Uninstalling mne-0.18.dev0:
      Successfully uninstalled mne-0.18.dev0


In [0]:
!git clone git://github.com/mne-tools/mne-python.git

Cloning into 'mne-python'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 98685 (delta 1), reused 0 (delta 0), pack-reused 98681
Receiving objects: 100% (98685/98685), 101.87 MiB | 35.40 MiB/s, done.
Resolving deltas: 100% (77665/77665), done.


In [0]:
!pip install matplotlib

In [0]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import random
import mne

In [0]:
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch

In [0]:
subject_number=4 


# load dataset in array
list_raw_fnames = [[0]*2]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,[1,2])

list_rawdata = np.zeros((subject_number,2), dtype='object')

for i in range(subject_number):
    for j in range(2):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)
print( list_rawdata[:][:])

Using default location ~/mne_data for EEGBCI...
Do you want to set the path:
    /root/mne_data
as the default EEGBCI dataset path in the mne-python config [y]/n? y
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
[........................................] 100.00% (  1.2 MB,  12.2 MB/s) \   
[........................................] 100.00% (  1.2 MB,  13.0 MB/s) /   
[........................................] 100.00% (  1.2 MB,  11.7 MB/s) |   
[........................................] 100.00% (  1.2 MB,  13.8 MB/s) -   
[........................................] 100.00% (  1.2 MB,  16.2 MB/s) \   
[........................................] 100.00% (  1.2 MB,  18.1 MB/s) /   
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Extracting EDF parameters fro

In [0]:
import math
task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 4 #step len

subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1
test_start_index=subject_img_number - math.floor(subject_img_number/4)

train_number =  test_start_index 
test_number = subject_img_number - test_start_index
train_number

1771

In [0]:
train_img = np.zeros((train_number*subject_number, 1, 65, 160), dtype = float)
train_label =[]

train_img.shape

test_img = np.zeros((test_number*subject_number, 1, 64, 160), dtype = float)
test_label =[]

train_img.shape

(7084, 1, 65, 160)

In [0]:

def normalize_channel_data(ch, list_std_for_ch, list_avg_for_ch):

    for i in range (64):
        
        ch[i] = (ch[i] - list_avg_for_ch[i]) / list_std_for_ch[i]
    #print(ch)
    return ch

In [0]:

for s in range(subject_number):
    
    rawdataChannels_t, times_t=rawdataChannels, times =list_rawdata[s][0][:64,:9600]
    list_std_for_ch =[]
    list_avg_for_ch =[]
    for j in range (64):
        #std for each channel
        std_ch = np.std(rawdataChannels_t[j])
        list_std_for_ch = np.append(list_std_for_ch, (std_ch))
        
        #average for each channel
        avg_ch = np.average(rawdataChannels_t[j])
        list_avg_for_ch = np.append(list_avg_for_ch, (avg_ch))
    
    for i in range(subject_img_number):
        rawdataChannels, times=rawdataChannels, times =list_rawdata[s][0][:64,0+i*sample_shift:sampel_number_per_sec+i*sample_shift]
        
        rawdataChannels = normalize_channel_data(rawdataChannels, list_std_for_ch, list_avg_for_ch)
        #print(rawdataChannels)
        
        if i<test_start_index :
            ids =  np.zeros((160), dtype = float)
            ids[:] =s
            rawdataChannels_id = np.vstack([rawdataChannels, ids])
        
            train_img[s*train_number + i][0] = rawdataChannels_id
            
            
        else:
            test_img[s*test_number + i-test_start_index][0] = rawdataChannels
            test_label = np.append(test_label, (s))
            
            


In [0]:
#shaffle train img array
train_img_shuffle = train_img

np.random.shuffle(train_img_shuffle) 


In [0]:
train_img_2 = np.zeros((train_number*subject_number, 1, 64, 160), dtype = float)

In [0]:
# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][64][0] )) 
    train_img_2[i][0] = np.delete(train_img_shuffle[i][0], (64), axis=0)

In [0]:

import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [0]:

# valid & train
x_train =train_img_2[:5000]
y_train =train_label[:5000]

x_valid =train_img_2[5000:]
y_valid =train_label[5000:]

x_train.shape


(5000, 1, 64, 160)

In [0]:
y_train[:10]

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [0]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (5,5), activation = 'relu', input_shape = (1,64,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (5,5), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

Instructions for updating:
Colocations handled automatically by placer.
(?, 64, 60, 156)
(?, 32, 30, 156)
(?, 28, 26, 128)
(?, ?)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 512)
(?, 4)


In [0]:
from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [0]:
history = model.fit(
        x_train,
        y_train,
        epochs = 5,
        batch_size = 20,
        validation_data = (x_valid, y_valid)
)

Instructions for updating:
Use tf.cast instead.
Train on 5000 samples, validate on 2084 samples
Epoch 1/5
5000/5000 [==============================] - 428s 86ms/step - loss: 0.5496 - acc: 0.7976 - val_loss: 0.0328 - val_acc: 0.9928
Epoch 2/5
5000/5000 [==============================] - 430s 86ms/step - loss: 0.0296 - acc: 0.9922 - val_loss: 0.0084 - val_acc: 0.9990
Epoch 3/5
5000/5000 [==============================] - 427s 85ms/step - loss: 0.0098 - acc: 0.9974 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 4/5
5000/5000 [==============================] - 428s 86ms/step - loss: 0.0070 - acc: 0.9978 - val_loss: 6.0151e-05 - val_acc: 1.0000
Epoch 5/5
5000/5000 [==============================] - 430s 86ms/step - loss: 0.0020 - acc: 0.9996 - val_loss: 0.0028 - val_acc: 0.9986
